In [52]:
import nltk
import re
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [53]:
messages = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t', names=['labels','message'])

# Data cleaning and pre-processing:

In [9]:
lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)


# Creating the TF-IDF model:

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=1250)
X = tf.fit_transform(corpus).toarray()

# Get Dummy Variables:

In [58]:
Y = pd.get_dummies(messages['labels'])

In [60]:
#To keep only 1 column:
Y = Y.iloc[:,1].values

# Train-Test Split:

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier

In [67]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, Y_train)

Y_pred=spam_detect_model.predict(X_test)

# Confusion Matrix:

In [71]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test,Y_pred)
confusion_matrix

array([[953,   2],
       [ 21, 139]], dtype=int64)

# Accuracy:

In [75]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test,Y_pred)
accuracy

0.979372197309417